## spark read s3 json

In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window, Row, SparkSession

import psycopg2
import pprint
import boto3
import json
import sys
import os

pp = pprint.PrettyPrinter(indent = 3)
print('imported modules.')

# Set Java home environment variable
# os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/temurin-8.jdk/Contents/Home'  # Update this path to match your Java installation

# read creds.json
with open("creds.json", "r") as f:
    creds = json.load(f)
    f.close()

imported modules.


## Spark

In [2]:
# Stop any existing Spark session
if 'spark' in locals():
    spark.stop()

In [3]:
try:
    # Create Spark session with required configurations
    spark = SparkSession.builder \
        .appName("YelpAnalysis") \
        .master("spark://spark-master:7077") \
        .config("spark.driver.memory", "2g") \
        .config("spark.executor.memory", "2g") \
        .config("spark.executor.cores", "4") \
        .config("spark.worker.memory", "2g") \
        .config("spark.cores.max", "4") \
        .config("spark.hadoop.fs.s3a.access.key", creds["aws_client"]) \
        .config("spark.hadoop.fs.s3a.secret.key", creds["aws_secret"]) \
        .config("spark.jars.packages", 
                "org.apache.hadoop:hadoop-aws:3.3.4," + 
                "org.apache.hadoop:hadoop-common:3.3.4," +
                "org.apache.hadoop:hadoop-aws:3.3.4," + 
                "com.amazonaws:aws-java-sdk-bundle:1.12.261," +
                "org.apache.logging.log4j:log4j-slf4j-impl:2.17.2," +
                "org.apache.logging.log4j:log4j-api:2.17.2," +
                "org.apache.logging.log4j:log4j-core:2.17.2," + 
                "org.apache.hadoop:hadoop-client:3.3.4," + 
                "io.delta:delta-core_2.12:2.4.0," + 
                "org.postgresql:postgresql:42.2.18") \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
        .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .getOrCreate()
    
        # .config("spark.jars.packages", "org.apache.hadoop:hadoop-common:3.3.4,org.apache.hadoop:hadoop-aws:3.3.4,org.apache.hadoop:hadoop-client:3.3.4,io.delta:delta-core_2.12:2.3.0,org.postgresql:postgresql:9.4.1212") \
        
    
except Exception as e:
    print(str(e))

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
org.apache.logging.log4j#log4j-slf4j-impl added as a dependency
org.apache.logging.log4j#log4j-api added as a dependency
org.apache.logging.log4j#log4j-core added as a dependency
org.apache.hadoop#hadoop-client added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ecbcb3f1-dee1-4f67-aa66-3dab5960fd59;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found org.apache.hadoop#hadoop-common;3.3.4 in central
	

## spark read json

In [5]:
bucket = "yelp-stevenhurwitt-2"
file = "yelp_academic_dataset_business.json"

# Define schema for Yelp business data
business_schema = StructType([
    StructField("business_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("address", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True),
    StructField("postal_code", StringType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("stars", DoubleType(), True),
    StructField("review_count", IntegerType(), True),
    StructField("is_open", IntegerType(), True),
    StructField("attributes", MapType(StringType(), StringType()), True),
    StructField("categories", StringType(), True),
    StructField("hours", MapType(StringType(), StringType()), True)
])

# Read JSON file from S3
business_df = spark.read \
    .format("json") \
    .schema(business_schema) \
    .option("multiLine", "false") \
    .load(f"s3a://{bucket}/{file}")


25/08/04 15:24:07 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


## verify data load

In [6]:
# Verify the data load
print("Row count:", business_df.count())
business_df.show(5, truncate=False)

Row count: 150346
+----------------------+------------------------+-------------------------------+-------------+-----+-----------+----------+------------+-----+------------+-------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------+
|business_id           |name                    |address                        |cit

In [7]:

# Schemas for yelp json datasets
business_schema = StructType([
    StructField("business_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("address", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True),
    StructField("postal_code", StringType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("stars", DoubleType(), True),
    StructField("review_count", IntegerType(), True),
    StructField("is_open", IntegerType(), True),
    StructField("attributes", MapType(StringType(), StringType()), True),
    StructField("categories", StringType(), True),
    StructField("hours", MapType(StringType(), StringType()), True)
])

users_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("review_count", IntegerType(), True),
    StructField("yelping_since", StringType(), True),
    StructField("useful", IntegerType(), True),
    StructField("funny", IntegerType(), True),
    StructField("cool", IntegerType(), True),
    StructField("elite", StringType(), True),
    StructField("friends", StringType(), True),
    StructField("fans", IntegerType(), True),
    StructField("average_stars", DoubleType(), True),
    StructField("compliment_hot", IntegerType(), True),
    StructField("compliment_more", IntegerType(), True),
    StructField("compliment_profile", IntegerType(), True),
    StructField("compliment_cute", IntegerType(), True),
    StructField("compliment_list", IntegerType(), True),
    StructField("compliment_note", IntegerType(), True),
    StructField("compliment_plain", IntegerType(), True),
    StructField("compliment_cool", IntegerType(), True),
    StructField("compliment_funny", IntegerType(), True),
    StructField("compliment_writer", IntegerType(), True),
    StructField("compliment_photos", IntegerType(), True)
])

checkins_schema = StructType([
    StructField("business_id", StringType(), True),
    StructField("date", StringType(), True)
])

reviews_schema = StructType([
    StructField("review_id", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("business_id", StringType(), True),
    StructField("stars", DoubleType(), True),
    StructField("useful", IntegerType(), True),
    StructField("funny", IntegerType(), True),
    StructField("cool", IntegerType(), True),
    StructField("text", StringType(), True),
    StructField("date", StringType(), True)
])

tips_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("business_id", StringType(), True),
    StructField("text", StringType(), True),
    StructField("date", StringType(), True),
    StructField("compliment_count", IntegerType(), True)
])


def read_json(path: str, schema: StructType) -> DataFrame:
    """
    Read a JSON file from S3 path with a specified schema.
    
    Args:
        path (str): S3 path to JSON file.
        schema (StructType): Spark DataFrame schema.
        
    Returns:
        DataFrame: Spark DataFrame containing the JSON data.
    """
    try:
        df = spark.read.json(path, schema=schema, multiLine=False)
        print(f"Successfully read JSON file from: {path}")
        print(f"Number of rows: {df.count()}")
        return df
    except Exception as e:
        print(f"Error reading JSON file from {path}")
        print(f"Error: {str(e)}")
        return None


## read json files

In [8]:
bucket = "yelp-stevenhurwitt-2"

# Read all json files
business_file = read_json(f"s3a://{bucket}/yelp_academic_dataset_business.json", business_schema)
checkin_file = read_json(f"s3a://{bucket}/yelp_academic_dataset_checkin.json", checkins_schema)
review_file = read_json(f"s3a://{bucket}/yelp_academic_dataset_review.json", reviews_schema)
tip_file = read_json(f"s3a://{bucket}/yelp_academic_dataset_tip.json", tips_schema)
user_file = read_json(f"s3a://{bucket}/yelp_academic_dataset_user.json", users_schema)

# Verify data loaded successfully
for df, name in [(business_file, "business"), 
                 (checkin_file, "checkins"),
                 (review_file, "reviews"),
                 (tip_file, "tips"),
                 (user_file, "users")]:
    if df is not None:
        print(f"\n{name} table schema:")
        df.printSchema()

# Business data
print("business data: ")
business_file.show(20)


Successfully read JSON file from: s3a://yelp-stevenhurwitt-2/yelp_academic_dataset_business.json


Number of rows: 150346
Successfully read JSON file from: s3a://yelp-stevenhurwitt-2/yelp_academic_dataset_checkin.json


Number of rows: 131930
Successfully read JSON file from: s3a://yelp-stevenhurwitt-2/yelp_academic_dataset_review.json


Number of rows: 6990280
Successfully read JSON file from: s3a://yelp-stevenhurwitt-2/yelp_academic_dataset_tip.json


Number of rows: 908915
Successfully read JSON file from: s3a://yelp-stevenhurwitt-2/yelp_academic_dataset_user.json


Number of rows: 1987897

business table schema:
root
 |-- business_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- stars: double (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- is_open: integer (nullable = true)
 |-- attributes: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- categories: string (nullable = true)
 |-- hours: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)


checkins table schema:
root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)


reviews table schema:
root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id: string (nulla

25/08/04 16:00:17 ERROR TaskSchedulerImpl: Lost executor 0 on 172.18.0.6: worker lost
25/08/04 16:39:41 ERROR TaskSchedulerImpl: Lost executor 1 on 172.18.0.5: worker lost


## read delta

In [4]:
def read_delta(path: str) -> DataFrame:
    """
    Read a Delta table from S3 path
    
    Args:
        path (str): S3 path to delta table
        
    Returns:
        DataFrame: Spark DataFrame containing the delta table data
    """
    try:
        df = spark.read \
            .format("delta") \
            .option("inferSchema", "true") \
            .load(path)
            
        print(f"Successfully read delta table from: {path}")
        print(f"Number of rows: {df.count()}")
        return df
        
    except Exception as e:
        print(f"Error reading delta table from {path}")
        print(f"Error: {str(e)}")
        return None

In [5]:
# Example usage:
bucket = "yelp-stevenhurwitt-2"

# Read all delta tables
business_file = read_delta(f"s3a://{bucket}/business")
checkin_file = read_delta(f"s3a://{bucket}/checkins")
review_file = read_delta(f"s3a://{bucket}/reviews")
tip_file = read_delta(f"s3a://{bucket}/tips")
user_file = read_delta(f"s3a://{bucket}/users")

# Verify data loaded successfully
for df, name in [(business_file, "business"), 
                 (checkin_file, "checkins"),
                 (review_file, "reviews"),
                 (tip_file, "tips"),
                 (user_file, "users")]:
    if df is not None:
        print(f"\n{name} table schema:")
        df.printSchema()

25/08/04 17:32:11 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


Successfully read delta table from: s3a://yelp-stevenhurwitt-2/business


25/08/04 17:32:18 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Number of rows: 150346
Successfully read delta table from: s3a://yelp-stevenhurwitt-2/checkins


Number of rows: 131930


Successfully read delta table from: s3a://yelp-stevenhurwitt-2/reviews


Number of rows: 6990280
Successfully read delta table from: s3a://yelp-stevenhurwitt-2/tips


Number of rows: 908915
Successfully read delta table from: s3a://yelp-stevenhurwitt-2/users
Number of rows: 1987897

business table schema:
root
 |-- business_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- stars: double (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- is_open: integer (nullable = true)
 |-- attributes: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- categories: string (nullable = true)
 |-- hours: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)


checkins table schema:
root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)


reviews table schema:
root
 |-- review_id: str

## business

In [22]:
business_file.show(20)

+--------------------+--------------------+--------------------+-------------------+-----+-----------+-------------+--------------+-----+------------+-------+--------------------+--------------------+--------------------+
|         business_id|                name|             address|               city|state|postal_code|     latitude|     longitude|stars|review_count|is_open|          attributes|          categories|               hours|
+--------------------+--------------------+--------------------+-------------------+-----+-----------+-------------+--------------+-----+------------+-------+--------------------+--------------------+--------------------+
|lwItZ1Ck3KtpCgG4C...|Stomel Elliot Att...|  532 Rte 70 W, Fl 2|        Cherry Hill|   NJ|      08002|    39.915478|    -75.016973|  5.0|           5|      1|                null|DUI Law, Professi...|{Monday -> 0:0-0:...|
|8rb-3VYXE37IZix4y...|Sharky's Sports B...|820 N Black Horse...|       Williamstown|   NJ|      08094|   39.6968

## checkins

In [23]:
checkin_file.show(20)

+--------------------+--------------------+
|         business_id|                date|
+--------------------+--------------------+
|k9-r1j1VeWY1BLS0I...|2011-10-02 14:21:...|
|k91vKZa8oDHGFEumL...|2012-05-05 23:00:...|
|k920l7wHSjLZDJLvn...|2016-05-07 18:28:...|
|k93U5RS4ohNqk9912...|2013-09-21 17:45:...|
|k94QZmoxe9RwUgwu1...|2016-05-27 14:21:...|
|k97mvU1TQ0Inr5vJ1...|2013-06-15 16:18:...|
|k99d0o0T_qS8-oidI...|2013-07-15 14:07:...|
|k9A4tAmei12tLprkr...|2011-01-21 16:04:...|
|k9ASnJ6A-nBZhdwPN...|2011-01-19 22:19:...|
|k9AnxGfiuHdB5DZo-...|2011-01-18 14:34:...|
|k9CwhDTFoInTVUvmv...|2011-04-02 22:18:...|
|k9H5CerrIu4tSiPQR...|2014-02-23 19:55:...|
|k9HFcXSsqMPy29Lt_...|2014-07-12 23:37:...|
|k9HrEbKuLHDc2rzxZ...|2012-12-08 21:08:...|
|k9KqXzZByboldRrAs...|2018-03-01 19:12:...|
|k9Ms586e_elwkuSfk...|2010-07-04 00:01:...|
|k9QaQYOU-egM3UpS8...|2010-11-13 15:52:...|
|k9SrdFwfKDQt-uLic...|2013-07-15 11:43:...|
|k9TX0DT-WNz2S741t...|2011-07-01 03:33:...|
|k9TyrHMGojZvQznyG...|2010-11-02

## users

In [24]:
user_file.show(20)

+--------------------+---------+------------+-------------------+--------------------+------+-----+----+----+-----+-------------+--------------+---------------+------------------+---------------+---------------+---------------+----------------+---------------+----------------+-----------------+-----------------+
|             user_id|     name|review_count|      yelping_since|             friends|useful|funny|cool|fans|elite|average_stars|compliment_hot|compliment_more|compliment_profile|compliment_cute|compliment_list|compliment_note|compliment_plain|compliment_cool|compliment_funny|compliment_writer|compliment_photos|
+--------------------+---------+------------+-------------------+--------------------+------+-----+----+----+-----+-------------+--------------+---------------+------------------+---------------+---------------+---------------+----------------+---------------+----------------+-----------------+-----------------+
|XACigsMQP4VYX970e...|       Al|           3|2019-06-30 19

## reviews

In [14]:
review_file.show(20)

+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+-------------------+----+-----+
|           review_id|             user_id|         business_id|stars|useful|funny|cool|                text|               date|year|month|
+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+-------------------+----+-----+
|dlIO7e_OGH0rjZ2n6...|GB3qwSzhx2d8DmDSI...|ecI3FBTM0f99Fnml3...|  5.0|     0|    0|   0|One of my all tim...|2006-05-19 02:22:45|2006|    5|
|p8gMItBQKZO4ka_K-...|FBRjdSizGuMyxQuSS...|fOhnSqmO4XY5vSI8w...|  3.0|     2|    0|   0|a light, airy atm...|2006-05-28 08:28:17|2006|    5|
|wtFktH-lJsZFcD9Ye...|6OV_PFTl9RW2FmYQo...|udHVIrP8z10Y4M0oc...|  5.0|     1|    0|   0|Great place for p...|2006-05-07 02:16:44|2006|    5|
|Pv7IhPgwadOpgKIad...|gfQqQYI5_hCAGEHlH...|Rrd1WEcFWYRH85HdH...|  3.0|     0|    1|   0|The bagels are pr...|2006-05-04 06:16:17|2006|    5|
|p_MRUqC20YmV

## join reviews and user_name and business_name

In [37]:
jdbc_url = "jdbc:postgresql://" + creds["postgres_host"] + ":5433/" + creds["postgres_db"]

connection_properties = {
    "user": creds["postgres_user"],        # PostgreSQL username
    "password": creds["postgres_password"], # PostgreSQL password
    "driver": "org.postgresql.Driver"       # JDBC driver class
}

join_query = """
    SELECT 
        r.*,
        u.name as user_name,
        b.name as business_name
    FROM reviews r
    LEFT JOIN yelp_users u ON r.user_id = u.user_id 
    LEFT JOIN business b ON r.business_id = b.business_id
"""

# Execute query using Spark's JDBC connection
df_reviews_join = spark.read \
    .jdbc(url=jdbc_url,
          table=f"({join_query}) as reviews_join",
          properties=connection_properties)

# df_reviews_join.show()

## spark sql

In [6]:
# Create temporary views to enable Spark SQL queries
review_file.createOrReplaceTempView("reviews")
user_file.createOrReplaceTempView("users")
business_file.createOrReplaceTempView("business")

# Execute the join using Spark SQL on the views
df_reviews_join = spark.sql("""
    SELECT 
        r.*,
        u.name as user_name,
        b.name as business_name
    FROM reviews r
    LEFT JOIN users u ON r.user_id = u.user_id 
    LEFT JOIN business b ON r.business_id = b.business_id
""")

df_reviews_join.show(20)

+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+-------------------+----+-----+---------+--------------------+
|           review_id|             user_id|         business_id|stars|useful|funny|cool|                text|               date|year|month|user_name|       business_name|
+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+-------------------+----+-----+---------+--------------------+
|wtFktH-lJsZFcD9Ye...|6OV_PFTl9RW2FmYQo...|udHVIrP8z10Y4M0oc...|  5.0|     1|    0|   0|Great place for p...|2006-05-07 02:16:44|2006|    5|    Stacy|            PetSmart|
|9VW3RXiC9B0Tjhacb...|6OV_PFTl9RW2FmYQo...|6qiBFWh4zQXTcac4K...|  3.0|     0|    0|   0|Interesting locat...|2006-05-07 00:25:00|2006|    5|    Stacy|Hollywood Bar & F...|
|os6IPZD4HuPqCDYIb...|6OV_PFTl9RW2FmYQo...|RQX-UJUzYwuNYHepp...|  4.0|     4|    0|   0|Roberts is locate...|2006-05-10 09:25:49|2006|    5|

In [ ]:
df_reviews_join.groupBy("year").count().show()

ERROR:root:KeyboardInterrupt while sending command.>               (0 + 0) / 25]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/local/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

## tips

In [29]:
tip_file.show(20)

+--------------------+--------------------+--------------------+-------------------+----------------+----+
|             user_id|         business_id|                text|               date|compliment_count|year|
+--------------------+--------------------+--------------------+-------------------+----------------+----+
|veuUOGS0bbOeQzu71...|aurSXIlX86Ob94kYv...|             Awesome|2021-01-01 23:19:41|               0|2021|
|JfAqGalRYKo3Byw73...|mP9dVul2VKgVIs_kZ...|SO quick and easy...|2021-01-21 03:03:02|               0|2021|
|ojvZZ_ZWRlfJDJN_k...|J7-mw216H21rjz-E7...|Love Kenny's flow...|2021-01-25 03:52:12|               0|2021|
|77xznhdIIfltTaY5j...|m3NH0HumRCp4ARx8R...|Must try the chee...|2021-02-15 18:51:36|               0|2021|
|lvthTfCQGD0qaEk6j...|ruXD0lB3rq4FMvnQL...|They have a touch...|2021-02-15 20:18:45|               0|2021|
|rsopw45VdcXxw0zEY...|djeYYE2MIT36obh5m...|The one on McCarr...|2021-01-01 05:19:55|               0|2021|
|3wVSwirz80_gXRcIB...|TsZEKqs0wzP3WHn

In [7]:
tip_file.groupBy("year").count().orderBy("year").show()

+----+------+
|year| count|
+----+------+
|2009|   665|
|2010| 26712|
|2011| 83395|
|2012|110459|
|2013|107563|
|2014|109160|
|2015| 89686|
|2016| 94333|
|2017| 93909|
|2018| 67033|
|2019| 57646|
|2020| 32436|
|2021| 34993|
|2022|   925|
+----+------+



## spark sql

In [8]:
# Create temporary views for the DataFrames
business_file.createOrReplaceTempView("business")
checkin_file.createOrReplaceTempView("checkins")

# Perform the join using Spark SQL
joined_df = spark.sql("""
    SELECT 
        b.name as business_name,
        b.business_id,
        c.date as checkin_date
    FROM business b
    INNER JOIN checkins c 
        ON b.business_id = c.business_id
    ORDER BY b.name, c.date
""")

# Show results
print("Number of joined records:", joined_df.count())
joined_df.show(20, truncate=False)

Number of joined records: 131930


+---------------------------------------------------+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## reviews

In [ ]:
review_file.show(20)

+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+-------------------+----+-----+
|           review_id|             user_id|         business_id|stars|useful|funny|cool|                text|               date|year|month|
+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+-------------------+----+-----+
|dlIO7e_OGH0rjZ2n6...|GB3qwSzhx2d8DmDSI...|ecI3FBTM0f99Fnml3...|  5.0|     0|    0|   0|One of my all tim...|2006-05-19 02:22:45|2006|    5|
|p8gMItBQKZO4ka_K-...|FBRjdSizGuMyxQuSS...|fOhnSqmO4XY5vSI8w...|  3.0|     2|    0|   0|a light, airy atm...|2006-05-28 08:28:17|2006|    5|
|wtFktH-lJsZFcD9Ye...|6OV_PFTl9RW2FmYQo...|udHVIrP8z10Y4M0oc...|  5.0|     1|    0|   0|Great place for p...|2006-05-07 02:16:44|2006|    5|
|Pv7IhPgwadOpgKIad...|gfQqQYI5_hCAGEHlH...|Rrd1WEcFWYRH85HdH...|  3.0|     0|    1|   0|The bagels are pr...|2006-05-04 06:16:17|2006|    5|
|p_MRUqC20YmV

## write to postgres

In [6]:
from pyspark.sql import DataFrame
import psycopg2

def drop_postgres_tables():
    """Drop PostgreSQL tables in correct order using psycopg2"""
    try:
        # Create direct PostgreSQL connection
        conn = psycopg2.connect(
            host=creds["postgres_host"],
            port=5433,
            database=creds["postgres_db"],
            user=creds["postgres_user"],
            password=creds["postgres_password"]
        )
        
        # Create cursor
        cur = conn.cursor()
        
        # Tables in reverse dependency order
        tables = ["tips", "reviews", "checkins", "users", "business"]
        
        for table in tables:
            cur.execute(f"DROP TABLE IF EXISTS {table} CASCADE;")
            print(f"Dropped table: {table}")
            
        # Commit changes and close connections
        conn.commit()
        cur.close()
        conn.close()
        print("Successfully dropped all tables")
            
    except Exception as e:
        print(f"Error dropping tables: {str(e)}")

# Drop existing tables first
drop_postgres_tables()

Dropped table: tips
Dropped table: reviews
Dropped table: checkins
Dropped table: users
Dropped table: business
Successfully dropped all tables


In [7]:
def write_to_postgres(df, table_name):
    """Write DataFrame to PostgreSQL table"""
    
    # Get database credentials from docker-compose environment
    jdbc_url = "jdbc:postgresql://" + creds["postgres_host"] + ":5433/" + creds["postgres_db"]
    connection_properties = {
        "user": creds["postgres_user"],
        "password": creds["postgres_password"],
        "driver": "org.postgresql.Driver"
    }
    
    try:
        print(f"Writing {table_name} to PostgreSQL...")
        
        # Convert complex types for PostgreSQL compatibility
        if table_name == "business":
            df = df.withColumn("attributes", to_json("attributes")) \
                  .withColumn("hours", to_json("hours"))
        elif table_name == "yelp_users":
            # Check if columns are arrays before converting
            if "elite" in df.columns and df.schema["elite"].dataType.typeName() == "array":
                df = df.withColumn("elite", array_join("elite", ","))
            if "friends" in df.columns and df.schema["friends"].dataType.typeName() == "array":
                df = df.withColumn("friends", array_join("friends", ","))
        
        # Write to PostgreSQL
        df.write \
            .jdbc(url=jdbc_url,
                  table=table_name,
                  mode="overwrite",
                  properties=connection_properties)
        
        print(f"Successfully wrote {df.count()} rows to {table_name}")
        
    except Exception as e:
        print(f"Error writing to {table_name}: {str(e)}")
        # Print schema for debugging
        print("\nSchema of the DataFrame:")
        df.printSchema()

# Write all tables in correct order
tables_to_write = {
    "business": business_file,
    "yelp_users": user_file,
    "checkins": checkin_file,
    "reviews": review_file,
    "tips": tip_file
}

for table_name in ["business", "yelp_users", "checkins", "reviews", "tips"]:
    if table_name in tables_to_write:
        write_to_postgres(tables_to_write[table_name], table_name)

Writing business to PostgreSQL...


Successfully wrote 150346 rows to business
Writing yelp_users to PostgreSQL...


Successfully wrote 1987897 rows to yelp_users
Writing checkins to PostgreSQL...


Successfully wrote 131930 rows to checkins
Writing reviews to PostgreSQL...


Successfully wrote 6990280 rows to reviews
Writing tips to PostgreSQL...


Successfully wrote 908915 rows to tips


25/07/25 19:48:11 WARN HeartbeatReceiver: Removing executor 0 with no recent heartbeats: 218693 ms exceeds timeout 120000 ms
25/07/25 19:48:11 ERROR TaskSchedulerImpl: Lost executor 0 on 172.18.0.5: Executor heartbeat timed out after 218693 ms
25/07/25 19:48:11 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_90_41 !
25/07/25 19:48:11 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_18_48 !
25/07/25 19:48:11 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_66_22 !
25/07/25 19:48:11 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_90_32 !
25/07/25 19:48:11 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_42_6 !
25/07/25 19:48:11 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_114_42 !
25/07/25 19:48:11 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_14_43 !
25/07/25 19:48:11 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_90_44 !
25/0

## counts

### spark sql

In [12]:
# Create temporary views for the DataFrames
business_file.createOrReplaceTempView("business")
user_file.createOrReplaceTempView("users")
checkin_file.createOrReplaceTempView("checkins")
review_file.createOrReplaceTempView("reviews")
tip_file.createOrReplaceTempView("tips")

# Execute the count query using Spark SQL
df_counts = spark.sql("""
    SELECT 'business' as table_name, COUNT(*) as count FROM business
    UNION ALL
    SELECT 'users' as table_name, COUNT(*) as count FROM users
    UNION ALL
    SELECT 'checkins' as table_name, COUNT(*) as count FROM checkins
    UNION ALL
    SELECT 'reviews' as table_name, COUNT(*) as count FROM reviews
    UNION ALL
    SELECT 'tips' as table_name, COUNT(*) as count FROM tips
    ORDER BY table_name
""")

df_counts.show()

+----------+-------+
|table_name|  count|
+----------+-------+
|  business| 150346|
|  checkins| 131930|
|   reviews|6990280|
|      tips| 908915|
|     users|1987897|
+----------+-------+



### jdbc

In [10]:
jdbc_url = "jdbc:postgresql://" + creds["postgres_host"] + ":5433/" + creds["postgres_db"]

connection_properties = {
    "user": creds["postgres_user"],        # PostgreSQL username
    "password": creds["postgres_password"], # PostgreSQL password
    "driver": "org.postgresql.Driver"       # JDBC driver class
}

count_query = """
SELECT 
    table_name,
    count
FROM (
    SELECT 'business' as table_name, COUNT(*) as count FROM business
    UNION ALL
    SELECT 'yelp_users' as table_name, COUNT(*) as count FROM yelp_users
    UNION ALL
    SELECT 'checkins' as table_name, COUNT(*) as count FROM checkins
    UNION ALL
    SELECT 'reviews' as table_name, COUNT(*) as count FROM reviews
    UNION ALL
    SELECT 'tips' as table_name, COUNT(*) as count FROM tips
) counts
ORDER BY table_name
"""

# Execute query using Spark's JDBC connection
df_counts_jdbc = spark.read \
    .jdbc(url=jdbc_url,
          table=f"({count_query}) as counts",
          properties=connection_properties)

df_counts_jdbc.show()

+----------+-------+
|table_name|  count|
+----------+-------+
|  business| 150346|
|  checkins| 131930|
|   reviews|6990280|
|      tips| 908915|
|yelp_users|1987897|
+----------+-------+



## write to postgres

In [ ]:
def write_to_postgres(df, table_name, mode="overwrite"):
    """
    Write DataFrame to PostgreSQL
    """
    try:
        print(f"Writing {table_name} to PostgreSQL...")
        
        df.write \
            .format("jdbc") \
            .option("url", f"jdbc:postgresql://{creds['postgres_host']}:5433/{creds['postgres_db']}") \
            .option("driver", "org.postgresql.Driver") \
            .option("dbtable", table_name) \
            .option("user", creds["postgres_user"]) \
            .option("password", creds["postgres_pass"]) \
            .mode(mode) \
            .save()
            
        print(f"Successfully wrote {table_name} to PostgreSQL")
        
    except Exception as e:
        print(f"Error writing {table_name} to PostgreSQL: {str(e)}")

# Process each table
tables = {
    "business": business_file,
    "checkins": checkin_file,
    "reviews": review_file,
    "tips": tip_file,
    "users": user_file
}

# Write each table to PostgreSQL
for table_name, df in tables.items():
    if df is not None:
        # Convert map types to json strings for PostgreSQL compatibility
        if table_name == "business":
            df = df.withColumn("attributes", to_json("attributes")) \
                  .withColumn("hours", to_json("hours"))
        elif table_name == "users":
            df = df.withColumn("elite", array_join("elite", ",")) \
                  .withColumn("friends", array_join("friends", ","))
                  
        write_to_postgres(df, table_name)

# Verify row counts
for table_name in tables.keys():
    count_df = spark.read \
        .format("jdbc") \
        .option("url", f"jdbc:postgresql://{creds['postgres_host']}:5433/{creds['postgres_db']}") \
        .option("driver", "org.postgresql.Driver") \
        .option("dbtable", f"(SELECT COUNT(*) as count FROM {table_name}) tmp") \
        .option("user", creds["postgres_user"]) \
        .option("password", creds["postgres_pass"]) \
        .load()
    
    print(f"{table_name} row count: {count_df.collect()[0]['count']}")

# Top 25 Restaurants by Average Rating

In [26]:
# Create a query to find top restaurants in Virginia
top_restaurants = spark.sql("""
    SELECT 
        name,
        address,
        city,
        state,
        ROUND(stars, 2) as average_stars,
        review_count,
        categories
    FROM business
    WHERE categories LIKE '%Restaurants%'
    ORDER BY stars DESC, review_count DESC
""")

# Display results in a cleaner format
top_restaurants.show(25, truncate=False)

+----------------------------------------+----------------------------------------+--------------+-----+-------------+------------+----------------------------------------------------------------------------------------------------------------------------+
|name                                    |address                                 |city          |state|average_stars|review_count|categories                                                                                                                  |
+----------------------------------------+----------------------------------------+--------------+-----+-------------+------------+----------------------------------------------------------------------------------------------------------------------------+
|Blues City Deli                         |2438 McNair Ave                         |Saint Louis   |MO   |5.0          |991         |Delis, Bars, Restaurants, Nightlife, Pubs, American (Traditional), Sandwiches                     

In [27]:
top_restaurants.groupBy("state").count().orderBy("state").show()

+-----+-----+
|state|count|
+-----+-----+
|   AB| 2410|
|   AZ| 2671|
|   CA| 1161|
|   CO|    1|
|   DE|  961|
|   FL| 8731|
|   HI|    1|
|   ID| 1302|
|   IL|  983|
|   IN| 4150|
|   LA| 3640|
|   MO| 4247|
|   MT|    1|
|   NC|    1|
|   NJ| 3341|
|   NV| 1673|
|   PA|12641|
|   TN| 4352|
|  XMS|    1|
+-----+-----+

